Model Name: sentence-transformers/msmarco-MiniLM-L6-cos-v5

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

In [11]:
data = pd.read_csv('../CSV/preped_data.csv')

labels = pd.read_csv('../CSV/labels.csv')

In [12]:
data.shape

(9494, 12)

In [13]:
labels.shape

(220, 1)

In [14]:
model = SentenceTransformer("sentence-transformers/msmarco-MiniLM-L6-cos-v5")

In [15]:
#3 min
data['d_embendding'] = data['description'].apply(lambda x: model.encode(x)if isinstance(x, str) else None)
data['b_embendding'] = data['business_tags'].apply(lambda x: model.encode(x) if isinstance(x, str) else None)

In [16]:
labels['l_embendding'] = labels['label'].apply(lambda x: model.encode(x) if isinstance(x, str) else None)

In [17]:
description_similarity = []
business_tags_similarity = []

In [18]:
for i in range(data.shape[0]):
    if data['d_embendding'][i] is not None:
        description_similarity.append(model.similarity([data['d_embendding'][i]], labels['l_embendding'].tolist())[0])
    else:
        description_similarity.append([0] * len(labels))
    if data['b_embendding'][i] is not None:
        business_tags_similarity.append(model.similarity([data['b_embendding'][i]], labels['l_embendding'].tolist())[0])
    else:
        business_tags_similarity.append([0] * len(labels))

c:\Users\anama_1lv\anaconda3\Lib\site-packages\sentence_transformers\util.py:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  a = torch.tensor(a)


In [19]:
similarities = (np.array(description_similarity) + np.array(business_tags_similarity)) / 2

In [20]:
def get_top_labels(similarities, labels, top_n=3):
    top_indices = np.argsort(similarities, axis=1)[:, -top_n:][:, ::-1]
    top_labels = [[labels[i] for i in indices] for indices in top_indices]
    top_labels_with_scores = [[(labels[i], similarities[row_idx, i]) for i in indices] for row_idx, indices in enumerate(top_indices)]
    return top_labels_with_scores

In [21]:
similarities_array = np.vstack(similarities[:data.shape[0]])

In [22]:
data['top_labels'] = get_top_labels(similarities_array, labels['label'].tolist(), top_n=3)

In [23]:
data[['description', 'business_tags', 'top_labels']].to_csv('../CSV/labelled_data_asymetric.csv', index=False)